### Importing Libraries

In [1]:
# import sys

# !{sys.executable} -m pip install beautifulsoup4
# !{sys.executable} -m pip install lxml
# !{sys.executable} -m pip install html5lib
# !{sys.executable} -m pip install geopy
# !{sys.executable} -m pip install geocoder
# !{sys.executable} -m pip install uszipcode
# !{sys.executable} -m pip install time
# !{sys.executable} -m pip install jupyterthemes
# !{sys.executable} -m jt -t monokai

from bs4 import BeautifulSoup as bs
import os

import requests as rs
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


import folium
import geopy
import geocoder
from geopy.geocoders import Nominatim
from sklearn.cluster import KMeans
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors

from uszipcode import SearchEngine
from uszipcode import Zipcode

import time
# from jupyterthemes import jtplot
# jtplot.style(theme="monokai", context="notebook", ticks=True, grid=False)

### Gathering the Names of Places in Maricopa County, Arizona

In [2]:
# Specify the target URL
print('Collecting data from the Phoenix New Times website: \n\n')

# url = 'https://www.ciclt.net/sn/clt/capitolimpact/gw_ziplist.aspx?FIPS=04013'
url = 'https://www.phoenixnewtimes.com/news/all-24-cities-and-towns-in-maricopa-county-ranked-from-worst-to-first-6641342'

# Fetch raw HTML content with a GET request
contents = rs.get(url)

# Parse the HTML content
soup = bs(contents.content, "lxml")
# soup = bs(contents.content, 'html.parser')
# # Print the parsed data
# print(soup.prettify())

# Find the relevant content for our purposes
story_body = soup.find('div', class_='story-body')
# print(story_body.prettify())

all_strongs = story_body.find_all('strong')
flag = 3
place_list = []
place_list_data = []

for places in all_strongs:
    if flag%3 == 0:
        place_list.append(places.text)
    flag += 1

place_list_df = pd.DataFrame(place_list, columns=['Names'])
# display(place_list_df)
place_list_df.drop(place_list_df.tail(1).index, axis=0, inplace=True)
# display(place_list_df)
place_list_df[['Prefix', 'Place']] = place_list_df['Names'].str.split(')', expand=True, n=1)
# display(place_list_df)
place_list_df.drop(columns=['Names', 'Prefix'], axis=1, inplace=True)
display(place_list_df)


,Place
0,Youngtown
1,Gila Bend
2,El Mirage
3,Tolleson
4,Surprise
5,Guadalupe
6,Avondale
7,Buckeye
8,Carefree
9,Fountain Hills


### Finding the Coordinates and Median Income of the Locations

In [3]:
lat_lng_coords = None

# # loop until you get the coordinates
# while(lat_lng_coords is None):
#   g = geocoder.google('Phoenix, Arizona')
#   lat_lng_coords = g.latlng

# place_list_df = place_list_df.reindex(columns=['Place', 'Latitude', 'Longitude'])
# display(place_list_df)

flag = 2
temp_list = []
geocoded_places = pd.DataFrame(columns=['Place', 'Latitude', 'Longitude', 'Median_Income'])

for locations in place_list_df['Place']:
    search_location = locations + ', Arizona'
    g = geocoder.locationiq(search_location, key='**********************')
    lat_lng_coords = g.latlng
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    
    search = SearchEngine(simple_zipcode=True)
    res = search.by_coordinates(latitude, longitude, radius=30, returns=1)
    median_income = res[0].median_household_income
    
    geocoded_places = geocoded_places.append({'Place': locations, 
                                              'Latitude': latitude, 
                                              'Longitude': longitude, 
                                              'Median_Income': median_income}, ignore_index=True)

#     print(locations)
#     print(search_location)
#     print('Latitude: ', latitude)
#     print('Longitude: ', longitude)
#     print('Median Household Income: ', median_income)
    if flag%2==0:
        time.sleep(1)
    flag+=1

display(geocoded_places)

,Place,Latitude,Longitude,Median_Income
0,Youngtown,33.593730,-112.303326,33884
1,Gila Bend,32.947827,-112.716824,30242
2,El Mirage,33.613034,-112.324487,47237
3,Tolleson,33.450050,-112.259309,50066
4,Surprise,33.629227,-112.368019,70302
5,Guadalupe,33.363125,-111.962533,53022
6,Avondale,33.435598,-112.349602,44658
7,Buckeye,33.370275,-112.583867,68839
8,Carefree,33.822261,-111.918203,100338
9,Fountain Hills,33.611711,-111.717361,73608


### Generating a Map using Folium

In [4]:
# Create a Map instance for Phoenix
place = 'Phoenix, Arizona'
geo_address = Nominatim(user_agent="phoenix_explorer")
locate_address = geo_address.geocode(place)
map_phoenix = folium.Map(location=[locate_address.latitude, locate_address.longitude], 
                         zoom_start=9)

# Add markers
for _, row in geocoded_places.iterrows():
    label = '{}, ${}'.format(row.Place, row.Median_Income)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([row.Latitude, row.Longitude], radius=5, popup=label, 
                        color='#252191', fill=True, fill_color='#6ecedb', 
                        fill_opacity=0.95, parse_html=False).add_to(map_phoenix) 

# Generate the Map
map_phoenix

### Using the Foursquare API to return the top 100 venues within a 500 metre radius

In [5]:
# Foursquare API
explore_url_prefix = 'https://api.foursquare.com/v2/venues/explore'
CLIENT_ID = '********************************' # Put Your Client Id
CLIENT_SECRET = '****************************' # Put You Client Secret 
VERSION = '20190712'
LIMIT = 100

# Function to return nearby venues
def getNearbyVenues(names, latitudes, longitudes, radius=2000):
    
    venues_list=[]
    
    for name, lat, lng in zip(names, latitudes, longitudes):
        print('Collecting venues for: ',name)
        
        # create the API request URL
        url = '{}?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            explore_url_prefix, CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)
            
        # make the GET request
        results = rs.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([( name, lat, lng, v['venue']['name'], 
                             v['venue']['location']['lat'], 
                             v['venue']['location']['lng'], 
                             v['venue']['categories'][0]['name']) for v in results])

#     print("\nThe length of the Venues List is:", len(venues_list))
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
#     nearby_venues = pd.DataFrame(venues_list)
    nearby_venues.columns = ['Place', 'Place_Latitude', 'Place_Longitude', 'Venue', 
                             'Venue_Latitude', 'Venue_Longitude', 'Venue_Category']
    
    return(nearby_venues)


# Passing the places for which venues are to be found to the function
phoenix_venues = getNearbyVenues(names=geocoded_places['Place'], 
                                 latitudes=geocoded_places['Latitude'], 
                                 longitudes=geocoded_places['Longitude'], 
                                 radius=2000)

display(phoenix_venues)

# venues = results['response']['groups'][0]['items']
# display(venues)

# neighborhood_venues = json_normalize(venues)
# display(neighborhood_venues)

,Place,Place_Latitude,Place_Longitude,Venue,Venue_Latitude,Venue_Longitude,Venue_Category
0,Youngtown,33.593730,-112.303326,Brenda's Kitchen,33.603695,-112.302043,American Restaurant
1,Youngtown,33.593730,-112.303326,Starbucks,33.602403,-112.290236,Coffee Shop
2,Youngtown,33.593730,-112.303326,QuikTrip,33.605500,-112.302520,Convenience Store
3,Youngtown,33.593730,-112.303326,Nino's,33.600808,-112.287748,Mexican Restaurant
4,Youngtown,33.593730,-112.303326,Mighty Moo Ice Cream,33.594641,-112.299814,Ice Cream Shop
...,...,...,...,...,...,...,...
1386,Phoenix,33.448437,-112.074142,Roosevelt Square Apartments,33.459479,-112.076028,Residential Building (Apartment / Condo)
1387,Phoenix,33.448437,-112.074142,Margaret T Hance Park,33.462160,-112.072843,Park
1388,Phoenix,33.448437,-112.074142,Barrio Café Gran Reserva,33.457025,-112.089526,Mexican Restaurant
1389,Phoenix,33.448437,-112.074142,Gallo Blanco Cafe y Bar,33.455995,-112.060827,Mexican Restaurant


### One-Hot Encoding for the Type of Venue

In [6]:
phoenix_onehot = pd.get_dummies(phoenix_venues[['Venue_Category']], prefix="", prefix_sep="")
phoenix_onehot.insert(0, 'Place_Name', phoenix_venues['Place'])

display(phoenix_onehot.head(15))

venues_count, categories_count = phoenix_onehot.shape
categories_count -= 1
print("There are {} different venues and {} unique venue categories.".format(venues_count, categories_count))

,Place_Name,ATM,Accessories Store,Adult Education Center,Advertising Agency,Airport,Airport Service,American Restaurant,Antique Shop,Arcade,...,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Water Park,Whisky Bar,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Youngtown,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Youngtown,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Youngtown,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Youngtown,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Youngtown,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Youngtown,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,Youngtown,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Youngtown,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,Youngtown,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Youngtown,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


There are 1391 different venues and 228 unique venue categories.


### Grouping by Place with the mean frequency of occurrence of the different types of venues

In [7]:
phoenix_grouped = phoenix_onehot.groupby('Place_Name').mean().reset_index()
display(phoenix_grouped)

,Place_Name,ATM,Accessories Store,Adult Education Center,Advertising Agency,Airport,Airport Service,American Restaurant,Antique Shop,Arcade,...,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Water Park,Whisky Bar,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Avondale,0.000000,0.000000,0.00,0.00,0.022222,0.022222,0.022222,0.000000,0.00,...,0.000000,0.000000,0.022222,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000
1,Buckeye,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000
2,Carefree,0.000000,0.022222,0.00,0.00,0.022222,0.000000,0.066667,0.022222,0.00,...,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.022222,0.000000,0.000000,0.000000
3,Cave Creek,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.027027,0.027027,0.00,...,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000
4,Chandler,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,...,0.010000,0.020000,0.000000,0.00,0.00,0.01,0.000000,0.000000,0.000000,0.000000
5,El Mirage,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.085714,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000
6,Fountain Hills,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.022222,0.000000,0.00,...,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.022222,0.000000,0.000000,0.000000
7,Gila Bend,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.047619,0.000000,0.00,...,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000
8,Gilbert,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.033898,0.000000,0.00,...,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.016949,0.000000,0.000000,0.000000
9,Glendale,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.025974,0.000000,0.00,...,0.012987,0.012987,0.012987,0.00,0.00,0.00,0.012987,0.000000,0.000000,0.000000


### Returning the top 5 types of Venues for each Place

In [8]:
num_top_venues = 5
for neighborhood in phoenix_grouped['Place_Name']:
    print("------{}------".format(neighborhood))
    temp = phoenix_grouped[phoenix_grouped['Place_Name'] == neighborhood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

------ Avondale------
                  venue  freq
0    Mexican Restaurant  0.18
1   Rental Car Location  0.07
2  Fast Food Restaurant  0.07
3        Sandwich Place  0.04
4     Convenience Store  0.04


------ Buckeye------
                venue  freq
0  Mexican Restaurant  0.20
1         Pizza Place  0.10
2      Discount Store  0.10
3         Coffee Shop  0.05
4         Candy Store  0.05


------ Carefree------
                 venue  freq
0          Coffee Shop  0.09
1            Locksmith  0.07
2                 Café  0.07
3  American Restaurant  0.07
4   Italian Restaurant  0.04


------ Cave Creek------
                    venue  freq
0             Coffee Shop  0.11
1      Mexican Restaurant  0.08
2       Convenience Store  0.05
3               BBQ Joint  0.05
4  Furniture / Home Store  0.05


------ Chandler------
                venue  freq
0  Mexican Restaurant  0.06
1   Convenience Store  0.04
2          Taco Place  0.04
3         Pizza Place  0.03
4  Italian Restaurant  0.03

### Ranking the top 10 most common venues for each place

In [9]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

In [10]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Place']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Place'] = phoenix_grouped['Place_Name']

for ind in np.arange(phoenix_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(
        phoenix_grouped.iloc[ind, :], num_top_venues)

display(neighborhoods_venues_sorted)

,Place,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Avondale,Mexican Restaurant,Rental Car Location,Fast Food Restaurant,Convenience Store,Sandwich Place,Pharmacy,Shoe Store,Bar,Big Box Store,Fish & Chips Shop
1,Buckeye,Mexican Restaurant,Pizza Place,Discount Store,Home Service,Sandwich Place,Fast Food Restaurant,Bank,BBQ Joint,Café,Candy Store
2,Carefree,Coffee Shop,American Restaurant,Locksmith,Café,Bar,Italian Restaurant,Sculpture Garden,Irish Pub,Steakhouse,Food Court
3,Cave Creek,Coffee Shop,Mexican Restaurant,Diner,Convenience Store,Steakhouse,Furniture / Home Store,Bar,BBQ Joint,Ice Cream Shop,Rental Car Location
4,Chandler,Mexican Restaurant,Convenience Store,Taco Place,Sandwich Place,Discount Store,Bank,Fast Food Restaurant,Italian Restaurant,Pizza Place,Park
5,El Mirage,Mexican Restaurant,Convenience Store,Pizza Place,Video Store,Fast Food Restaurant,Pharmacy,Baseball Field,Food,Big Box Store,Thrift / Vintage Store
6,Fountain Hills,Pizza Place,Italian Restaurant,Coffee Shop,Mexican Restaurant,Bank,Park,Grocery Store,Pharmacy,Construction & Landscaping,Gym / Fitness Center
7,Gila Bend,Fast Food Restaurant,Mexican Restaurant,Hotel,Ice Cream Shop,Discount Store,Truck Stop,Memorial Site,American Restaurant,Pizza Place,Gas Station
8,Gilbert,Mexican Restaurant,Sandwich Place,Breakfast Spot,Cosmetics Shop,Ice Cream Shop,Brewery,Gym,Italian Restaurant,Coffee Shop,Sushi Restaurant
9,Glendale,Convenience Store,Mexican Restaurant,Pizza Place,Fast Food Restaurant,Discount Store,Park,Bank,American Restaurant,Food & Drink Shop,Pet Store


### Clustering Phoenix into 5 Clusters using K-Means Clustering

In [11]:
# Set number of clusters
k = 5

# Drop the neighborhood name column so that each column contains only the feature set.
phoenix_grouped_clustering = phoenix_grouped.drop('Place_Name', 1)

# Run k-means clustering
kmeans = KMeans(n_clusters=k, random_state=0).fit(phoenix_grouped_clustering)

# Check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

# Add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

phoenix_merged = geocoded_places

# Merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
phoenix_merged = phoenix_merged.join(neighborhoods_venues_sorted.set_index('Place'), on='Place')

# print(phoenix_merged.shape)
display(phoenix_merged.head(24))

(24, 15)


,Place,Latitude,Longitude,Median_Income,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Youngtown,33.593730,-112.303326,33884,0,Convenience Store,Fast Food Restaurant,Ice Cream Shop,Pharmacy,Coffee Shop,Mexican Restaurant,Fried Chicken Joint,Pet Store,Sandwich Place,Café
1,Gila Bend,32.947827,-112.716824,30242,3,Fast Food Restaurant,Mexican Restaurant,Hotel,Ice Cream Shop,Discount Store,Truck Stop,Memorial Site,American Restaurant,Pizza Place,Gas Station
2,El Mirage,33.613034,-112.324487,47237,2,Mexican Restaurant,Convenience Store,Pizza Place,Video Store,Fast Food Restaurant,Pharmacy,Baseball Field,Food,Big Box Store,Thrift / Vintage Store
3,Tolleson,33.450050,-112.259309,50066,1,Convenience Store,Mexican Restaurant,Intersection,Home Service,Burger Joint,Hotel,Fried Chicken Joint,Deli / Bodega,Sandwich Place,Seafood Restaurant
4,Surprise,33.629227,-112.368019,70302,0,Sandwich Place,Fast Food Restaurant,Baseball Field,Mobile Phone Shop,Hardware Store,Sushi Restaurant,Coffee Shop,Mexican Restaurant,Italian Restaurant,Department Store


### Visualizing the Clusters

In [13]:
# Create a map instance
map_phoenix = folium.Map(location=[locate_address.latitude, locate_address.longitude], 
                         zoom_start=9)

# Set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# Add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(phoenix_merged['Latitude'], 
                                  phoenix_merged['Longitude'], 
                                  phoenix_merged['Place'], 
                                  phoenix_merged['Cluster Labels']):
    
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker([lat, lon], radius=5, popup=label, 
                        color=rainbow[cluster-1], fill=True, 
                        fill_color=rainbow[cluster-1], fill_opacity=0.7).add_to(map_phoenix)
       
map_phoenix

#### Cluster 1

In [14]:
# cluster_1 = phoenix_merged.loc[phoenix_merged['Cluster Labels'] == 0, phoenix_merged.columns[ [2] + list(range(5, phoenix_merged.shape[1]))]]
cluster_1 = phoenix_merged.loc[phoenix_merged['Cluster Labels'] == 0]
print('Average Median Income for Cluster 1: $', round(cluster_1['Median_Income'].mean()), 2)
display(cluster_1)

Average Median Income for Cluster 1: $ 55462.0 2


,Place,Latitude,Longitude,Median_Income,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Youngtown,33.593730,-112.303326,33884,0,Convenience Store,Fast Food Restaurant,Ice Cream Shop,Pharmacy,Coffee Shop,Mexican Restaurant,Fried Chicken Joint,Pet Store,Sandwich Place,Café
4,Surprise,33.629227,-112.368019,70302,0,Sandwich Place,Fast Food Restaurant,Baseball Field,Mobile Phone Shop,Hardware Store,Sushi Restaurant,Coffee Shop,Mexican Restaurant,Italian Restaurant,Department Store
5,Guadalupe,33.363125,-111.962533,53022,0,Fast Food Restaurant,Sandwich Place,Hotel,Convenience Store,Mexican Restaurant,Gas Station,Coffee Shop,Video Store,Resort,Chinese Restaurant
9,Fountain Hills,33.611711,-111.717361,73608,0,Pizza Place,Italian Restaurant,Coffee Shop,Mexican Restaurant,Bank,Park,Grocery Store,Pharmacy,Construction & Landscaping,Gym / Fitness Center
10,Mesa,33.415112,-111.831479,36586,0,Mexican Restaurant,Convenience Store,Sandwich Place,Bank,Brewery,Hotel,Ice Cream Shop,Park,Bar,Museum
12,Gilbert,33.352763,-111.789037,75365,0,Mexican Restaurant,Sandwich Place,Breakfast Spot,Cosmetics Shop,Ice Cream Shop,Brewery,Gym,Italian Restaurant,Coffee Shop,Sushi Restaurant
13,Queen Creek,33.248386,-111.634158,73367,0,Mexican Restaurant,Pizza Place,Fast Food Restaurant,Furniture / Home Store,Pharmacy,Gym,Sandwich Place,Burger Joint,Baseball Field,Cosmetics Shop
16,Peoria,33.580612,-112.237294,45886,0,Fast Food Restaurant,Convenience Store,Video Store,Mexican Restaurant,Pizza Place,Hardware Store,Thrift / Vintage Store,Clothing Store,Chinese Restaurant,Theater
17,Litchfield Park,33.493380,-112.358124,64383,0,Coffee Shop,Grocery Store,Golf Course,Chinese Restaurant,Pizza Place,Park,Italian Restaurant,Mexican Restaurant,American Restaurant,Spa
18,Chandler,33.306160,-111.841250,56414,0,Mexican Restaurant,Convenience Store,Taco Place,Sandwich Place,Discount Store,Bank,Fast Food Restaurant,Italian Restaurant,Pizza Place,Park


In [15]:
# Visualizing the Cluster
map_phoenix = folium.Map(location=[locate_address.latitude, locate_address.longitude], 
                         zoom_start=9)

for _, row in cluster_1.iterrows():
    label = '{}, ${}'.format(row.Place, row.Median_Income)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([row.Latitude, row.Longitude], radius=5, popup=label, 
                        color='#252191', fill=True, fill_color='#6ecedb', 
                        fill_opacity=0.95, parse_html=False).add_to(map_phoenix) 

map_phoenix

#### Cluster 2

In [16]:
cluster_2 = phoenix_merged.loc[phoenix_merged['Cluster Labels'] == 1]
print('Average Median Income for Cluster 2: $', round(cluster_2['Median_Income'].mean()), 2)
display(cluster_2)

Average Median Income for Cluster 2: $ 50066.0 2


,Place,Latitude,Longitude,Median_Income,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Tolleson,33.45005,-112.259309,50066,1,Convenience Store,Mexican Restaurant,Intersection,Home Service,Burger Joint,Hotel,Fried Chicken Joint,Deli / Bodega,Sandwich Place,Seafood Restaurant


In [17]:
# Visualizing the Cluster
map_phoenix = folium.Map(location=[locate_address.latitude, locate_address.longitude], 
                         zoom_start=9)

for _, row in cluster_2.iterrows():
    label = '{}, ${}'.format(row.Place, row.Median_Income)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([row.Latitude, row.Longitude], radius=5, popup=label, 
                        color='#252191', fill=True, fill_color='#6ecedb', 
                        fill_opacity=0.95, parse_html=False).add_to(map_phoenix) 

map_phoenix

#### Cluster 3

In [18]:
cluster_3 = phoenix_merged.loc[phoenix_merged['Cluster Labels'] == 2]
print('Average Median Income for Cluster 3: $', round(cluster_3['Median_Income'].mean()), 2)
display(cluster_3)

Average Median Income for Cluster 3: $ 51348.0 2


,Place,Latitude,Longitude,Median_Income,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,El Mirage,33.613034,-112.324487,47237,2,Mexican Restaurant,Convenience Store,Pizza Place,Video Store,Fast Food Restaurant,Pharmacy,Baseball Field,Food,Big Box Store,Thrift / Vintage Store
6,Avondale,33.435598,-112.349602,44658,2,Mexican Restaurant,Rental Car Location,Fast Food Restaurant,Convenience Store,Sandwich Place,Pharmacy,Shoe Store,Bar,Big Box Store,Fish & Chips Shop
7,Buckeye,33.370275,-112.583867,68839,2,Mexican Restaurant,Pizza Place,Discount Store,Home Service,Sandwich Place,Fast Food Restaurant,Bank,BBQ Joint,Café,Candy Store
11,Goodyear,33.435367,-112.357601,44658,2,Mexican Restaurant,Fast Food Restaurant,Rental Car Location,Convenience Store,Pizza Place,Vietnamese Restaurant,Discount Store,Hotel,Sandwich Place,Bank


In [19]:
# Visualizing the Cluster
map_phoenix = folium.Map(location=[locate_address.latitude, locate_address.longitude], 
                         zoom_start=9)

for _, row in cluster_3.iterrows():
    label = '{}, ${}'.format(row.Place, row.Median_Income)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([row.Latitude, row.Longitude], radius=5, popup=label, 
                        color='#252191', fill=True, fill_color='#6ecedb', 
                        fill_opacity=0.95, parse_html=False).add_to(map_phoenix) 

map_phoenix

#### Cluster 4

In [20]:
cluster_4 = phoenix_merged.loc[phoenix_merged['Cluster Labels'] == 3]
print('Average Median Income for Cluster 4: $', round(cluster_4['Median_Income'].mean()), 2)
display(cluster_4)

Average Median Income for Cluster 4: $ 36308.0 2


,Place,Latitude,Longitude,Median_Income,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Gila Bend,32.947827,-112.716824,30242,3,Fast Food Restaurant,Mexican Restaurant,Hotel,Ice Cream Shop,Discount Store,Truck Stop,Memorial Site,American Restaurant,Pizza Place,Gas Station
15,Wickenburg,33.968096,-112.730135,42375,3,Fast Food Restaurant,Mexican Restaurant,Convenience Store,Sandwich Place,Dive Bar,Discount Store,American Restaurant,Hotel,Ice Cream Shop,Gas Station


In [21]:
# Visualizing the Cluster
map_phoenix = folium.Map(location=[locate_address.latitude, locate_address.longitude], 
                         zoom_start=9)

for _, row in cluster_4.iterrows():
    label = '{}, ${}'.format(row.Place, row.Median_Income)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([row.Latitude, row.Longitude], radius=5, popup=label, 
                        color='#252191', fill=True, fill_color='#6ecedb', 
                        fill_opacity=0.95, parse_html=False).add_to(map_phoenix) 

map_phoenix

#### Cluster 5

In [22]:
cluster_5 = phoenix_merged.loc[phoenix_merged['Cluster Labels'] == 4]
print('Average Median Income for Cluster 5: $', round(cluster_5['Median_Income'].mean()), 2)
display(cluster_5)

Average Median Income for Cluster 5: $ 67360.0 2


,Place,Latitude,Longitude,Median_Income,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Carefree,33.822261,-111.918203,100338,4,Coffee Shop,American Restaurant,Locksmith,Café,Bar,Italian Restaurant,Sculpture Garden,Irish Pub,Steakhouse,Food Court
14,Cave Creek,33.833333,-111.950833,88938,4,Coffee Shop,Mexican Restaurant,Diner,Convenience Store,Steakhouse,Furniture / Home Store,Bar,BBQ Joint,Ice Cream Shop,Rental Car Location
19,Paradise Valley,33.532429,-111.950512,109185,4,Spa,American Restaurant,Resort,Restaurant,Hotel Pool,Mountain,State / Provincial Park,Shipping Store,Business Service,Bar
21,Tempe,33.425506,-111.940012,30582,4,Coffee Shop,Breakfast Spot,Mexican Restaurant,Sandwich Place,Park,American Restaurant,Brewery,Pizza Place,Bar,Thai Restaurant
22,Scottsdale,33.494219,-111.926018,49111,4,Coffee Shop,American Restaurant,Mexican Restaurant,Bar,Pizza Place,Hotel,New American Restaurant,Taco Place,Italian Restaurant,Sushi Restaurant
23,Phoenix,33.448437,-112.074142,26008,4,Coffee Shop,American Restaurant,Hotel,Art Gallery,Pizza Place,Mexican Restaurant,Cocktail Bar,Music Venue,Bar,Salon / Barbershop


In [23]:
# Visualizing the Cluster
map_phoenix = folium.Map(location=[locate_address.latitude, locate_address.longitude], 
                         zoom_start=9)

for _, row in cluster_5.iterrows():
    label = '{}, ${}'.format(row.Place, row.Median_Income)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([row.Latitude, row.Longitude], radius=5, popup=label, 
                        color='#252191', fill=True, fill_color='#6ecedb', 
                        fill_opacity=0.95, parse_html=False).add_to(map_phoenix) 

map_phoenix